In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'
%matplotlib inline

import yelp_tfidf as lib_tfidf
from merge_main import open_pickle, save_to_pickle
import lda
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import CountVectorizer

/Users/tracy/anaconda/lib/python2.7/site-packages/fuzzywuzzy/fuzz.py:33: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
from IPython.display import display

# Phoenix, AZ -- Classification:

In [3]:
df_AZ = open_pickle('../data/phx/phoenix_yelp_features.pkl')

In [4]:
df_AZ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23487 entries, 2 to 37535
Data columns (total 21 columns):
business_id       23487 non-null object
id_               23487 non-null object
date              23487 non-null object
inspec_id         23487 non-null object
n_priority        23487 non-null object
grade             23487 non-null object
purpose           23487 non-null object
n_violations      16427 non-null float64
v_core            23487 non-null int64
sum_core          23487 non-null int64
v_foundation      23487 non-null int64
sum_foundation    23487 non-null int64
v_priority        23487 non-null int64
sum_priority      23487 non-null int64
date_start        23487 non-null datetime64[ns]
rev_ct            23487 non-null int64
neg_ct            23487 non-null int64
stars_avg         23487 non-null float64
rev_len_avg       23487 non-null float64
stars_var         23487 non-null float64
text              23487 non-null object
dtypes: datetime64[ns](1), float64(4), int64(8)

# Negative Words

In [165]:
reload(lib_tfidf)

<module 'yelp_tfidf' from 'yelp_tfidf.py'>

In [9]:
neg = pd.read_csv('../data/negative-words.txt', skiprows=35,header=None)
pos = pd.read_csv('../data/positive-words.txt', skiprows=35,header=None)
neg_stem = lib_tfidf.stem_tokens(neg.values.flatten().tolist())
pos_stem = lib_tfidf.stem_tokens(pos.values.flatten().tolist())

na�ve


In [6]:
print len(set(neg_stem))
print len(set(pos_stem))

3159
1391


In [97]:
set(neg_stem)

{u'broken-heart',
 u'haggl',
 u'lawbreak',
 u'hatr',
 u'brashli',
 u'poorli',
 u'protest',
 u'thirst',
 u'tenuous',
 u'hate',
 u'inexpert',
 u'defraud',
 u'skeleton',
 u'accus',
 u'spew',
 u'sorri',
 u'poorest',
 u'scold',
 u'self-destruct',
 u'forebod',
 u'deviat',
 u'fatcat',
 u'daze',
 u'sputter',
 u'unnatur',
 u'hedg',
 u'bulkier',
 u'lonesom',
 u'milit',
 u'lure',
 u'risk',
 u'appal',
 u'rebuk',
 u'lurk',
 u'distort',
 u'manic',
 u'smack',
 u'befoul',
 u'foul',
 u'grumpier',
 u'improperli',
 u'senil',
 u'disturb',
 u'traumat',
 u'unrest',
 u'pinch',
 u'roadblock',
 u'brutish',
 u'relentlessli',
 u'rusti',
 u'overdo',
 u'hord',
 u'constern',
 u'naiv',
 u'dogmat',
 u'enfeebl',
 u'louder',
 u'bedlamit',
 u'blasphemi',
 u'overdu',
 u'ploy',
 u'monster',
 u'pander',
 u'decri',
 u'wreck',
 u'ruthless',
 u'neg',
 u'poison',
 u'backach',
 u'gawki',
 u'sunder',
 u'blur',
 u'ugliest',
 u'blackmail',
 u'succumb',
 u'hazi',
 u'stern',
 u'avers',
 u'prate',
 u'deadbeat',
 u'bullyingli',
 u'smu

In [102]:
set(pos_stem)

{u'efficaci',
 u'pardon',
 u'pamperedli',
 u'saver',
 u'sleek',
 u'wondrou',
 u'encouragingli',
 u'profici',
 u'authorit',
 u'accur',
 u'enchantingli',
 u'overtak',
 u'outwit',
 u'regal',
 u'cushi',
 u'worth',
 u'kudo',
 u'compassion',
 u'lumin',
 u'affect',
 u'viewabl',
 u'flashi',
 u'cost-sav',
 u'enjoy',
 u'titil',
 u'sooth',
 u'empow',
 u'valiant',
 u'illumin',
 u'amus',
 u'liberti',
 u'hero',
 u'jubilantli',
 u'delect',
 u'feisti',
 u'galor',
 u'unforgett',
 u'precis',
 u'god-send',
 u'portabl',
 u'tremend',
 u'golden',
 u'envious',
 u'fortitud',
 u'moral',
 u'calm',
 u'recommend',
 u'catchi',
 u'exuberantli',
 u'solicit',
 u'relax',
 u'award',
 u'exce',
 u'holi',
 u'fast-grow',
 u'cheapest',
 u'elegantli',
 u'work',
 u'defer',
 u'goood',
 u'unassail',
 u'joyous',
 u'captiv',
 u'exquisit',
 u'assuredli',
 u'innocu',
 u'attract',
 u'concis',
 u'hot',
 u'well-regard',
 u'recoveri',
 u'verifi',
 u'energy-effici',
 u'confid',
 u'righten',
 u'modest',
 u'outshin',
 u'beauti',
 u'substa

In [7]:
%%time
tf_pos, tf_pos_vect = lib_tfidf.yelp_tf(df_AZ, 'text', vocab=list(set(pos_stem)))

CPU times: user 5min 49s, sys: 7.23 s, total: 5min 57s
Wall time: 6min 2s


In [8]:
%%time
tf_neg, tf_neg_vect = lib_tfidf.yelp_tf(df_AZ, 'text', vocab=list(set(neg_stem)))

CPU times: user 5min 44s, sys: 6.23 s, total: 5min 51s
Wall time: 5min 54s


In [9]:
#print top occuring positive words (only count document frequency)
doc_pos = (tf_pos > 0).sum(axis=0).tolist()[0]
ind_pos = np.argsort(doc_pos)[::-1]
vocab_pos = tf_pos_vect.get_feature_names()
top_pos = [vocab_pos[x] for x in ind_pos]

In [10]:
#print top occuring positive words (cumulative count across all docs)
ct_pos = tf_pos.sum(axis=0).tolist()[0]
ind_pos2 = np.argsort(ct_pos)[::-1]
vocab_pos2 = tf_pos_vect.get_feature_names()
top_pos2 = [vocab_pos2[x] for x in ind_pos2]

In [11]:
#print top occuring positive words (only count document frequency)
doc_neg = (tf_neg > 0).sum(axis=0).tolist()[0]
ind_neg = np.argsort(doc_neg)[::-1]
vocab_neg = tf_neg_vect.get_feature_names()
top_neg = [vocab_neg[x] for x in ind_neg]

In [12]:
#print top occuring positive words (cumulative count across all docs)
ct_neg = tf_neg.sum(axis=0).tolist()[0]
ind_neg2 = np.argsort(ct_neg)[::-1]
vocab_neg2 = tf_neg_vect.get_feature_names()
top_neg2 = [vocab_neg2[x] for x in ind_neg2]

In [13]:
set(top_pos[:203]) - set(top_pos2[:203])

{u'fortun', u'pleasur', u'safe', u'thrill'}

In [14]:
set(top_pos2[:203]) - set(top_pos[:203])

{u'golden', u'liberti', u'paradis', u'peach'}

In [15]:
[top_pos[x] for x in ind_pos[200:300]]

[u'effici',
 u'coven',
 u'eyecatch',
 u'fame',
 u'frugal',
 u'facilit',
 u'refund',
 u'vivid',
 u'fantast',
 u'commodi',
 u'frolic',
 u'blith',
 u'hospit',
 u'equit',
 u'loveli',
 u'advoc',
 u'enjoy',
 u'charisma',
 u'salutari',
 u'phenomen',
 u'beckon',
 u'courteou',
 u'geeki',
 u'shine',
 u'trouble-fre',
 u'assur',
 u'tickl',
 u'enthus',
 u'tougher',
 u'scenic',
 u'agreeabl',
 u'jollifi',
 u'convinc',
 u'propiti',
 u'glori',
 u'hilari',
 u'fertil',
 u'clear',
 u'record-set',
 u'unencumb',
 u'remuner',
 u'thought',
 u'fervidli',
 u'high-qual',
 u'avail',
 u'infal',
 u'illuminati',
 u'envi',
 u'privileg',
 u'goood',
 u'convienc',
 u'kid-friendli',
 u'engross',
 u'purifi',
 u'event',
 u'delici',
 u'unbound',
 u'carefre',
 u'consist',
 u'hug',
 u'judici',
 u'gutsi',
 u'supurb',
 u'well-receiv',
 u'unwav',
 u'reconcili',
 u'dote',
 u'premier',
 u'workabl',
 u'sociabl',
 u'outperform',
 u'fervid',
 u'simplest',
 u'thumb-up',
 u'smitten',
 u'commit',
 u'leviti',
 u'reliabl',
 u'lavish',
 u'

# LDA - Codeword

Steps:
- calculate term frequency matrix
- use top_pos and top_neg vectors (from above) to subset columns to +/- words
    * Add column GOODREVIEW = sum over counts in top_pos columns
    * Add column BADREVIEW = sum over counts in top_neg columns
- apply LDA (20 topics)

In [60]:
# def get_codewords(n, tf=tf, tf_vector=tf_vector, top_pos=top_pos, top_neg=top_neg):
def get_codewords(n, tf, tf_vector, top_pos, top_neg):
    # Find index of top 300 positive and negative vocabulary columns:
    tf_vocab = tf_vector.get_feature_names()
    ind_tf_pos = [tf_vocab.index(x) for x in top_pos[:n] if x in tf_vocab]
    ind_tf_neg = [tf_vocab.index(x) for x in top_neg[:n] if x in tf_vocab]
    print len(ind_tf_pos)
    print len(ind_tf_neg)
    
    # Create new features: GOODREVIEW and BADREVIEW
    GOODREVIEW = tf[:,ind_tf_pos].sum(axis=1)
    BADREVIEW = tf[:,ind_tf_neg].sum(axis=1)
    # exclude additional stop words:
    ind = [x not in ['wa','thi'] for x in tf_vocab]

    # Create final feature matrix (and update vocabulary)
    tf_final = hstack((tf[:,np.array(ind).nonzero()[0]], csr_matrix(GOODREVIEW), csr_matrix(BADREVIEW)))
    vocab_final = [tf_vocab[i] for i, x in enumerate(ind) if x] + ['GOODREVIEW','BADREVIEW']

    return tf_final, vocab_final

def fit_and_print(tf_final, vocab_final, alpha=0.1, eta=0.01):
    # Fit LDA
    model_lda = lda.LDA(n_topics=20, n_iter=500, random_state=981, alpha=alpha, eta=eta)
    model_lda.fit(tf_final)
    # Top 10 words
    n = 10
    topic_word = model_lda.topic_word_
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab_final)[np.argsort(topic_dist)][:-(n+1):-1]
        print '*Topic %s\n- %s' % (i, ' '.join(topic_words))
        
    return model_lda

def print_topics(topic_word, vocab_final, n=10):
    prob_score = get_probability_score(topic_word)
    for i, topic_dist in enumerate(prob_score):
        topic_words = np.array(vocab_final)[np.argsort(topic_dist)][:-(n+1):-1]
        print '*Topic %s\n- %s' % (i, ' '.join(topic_words))
    

In [53]:
df_AZ_clean = df_AZ[df_AZ.text<>'.']

In [56]:
%%time
# Calculate term frequency matrix
tf, tf_vector = lib_tfidf.yelp_tf(df_AZ_clean, 'text', max_features=10000)

CPU times: user 5min 34s, sys: 7.3 s, total: 5min 41s
Wall time: 5min 49s


In [17]:
# Find index of top 300 positive and negative vocabulary columns:
tf_vocab = tf_vector.get_feature_names()
ind_tf_pos = [tf_vocab.index(x) for x in top_pos[:300] if x in tf_vocab]
ind_tf_neg = [tf_vocab.index(x) for x in top_neg[:300] if x in tf_vocab]
print len(ind_tf_pos)
print len(ind_tf_neg)

300
300


In [38]:
# Create new features: GOODREVIEW and BADREVIEW
GOODREVIEW = tf[:,ind_tf_pos].sum(axis=1)
BADREVIEW = tf[:,ind_tf_neg].sum(axis=1)
# exclude additional stop words:
ind = [x not in ['wa','thi'] for x in tf_vocab]

# Create final feature matrix (and update vocabulary)
tf_final = hstack((tf[:,np.array(ind).nonzero()[0]], csr_matrix(GOODREVIEW), csr_matrix(BADREVIEW)))
vocab_final = [tf_vocab[i] for i, x in enumerate(ind) if x] + ['GOODREVIEW','BADREVIEW']

In [39]:
%%time
# Fit LDA
model = lda.LDA(n_topics=20, n_iter=500, random_state=981)
model.fit(tf_final)

CPU times: user 22min 40s, sys: 13.4 s, total: 22min 53s
Wall time: 23min 10s


In [60]:
%%time
tf_final_300, vocab_final_300 = get_codewords(300)
model_300 = fit_and_print(tf_final_300, vocab_final_300)

300
300
*Topic 0
- GOODREVIEW sandwich salad BADREVIEW good place lunch bread great veri
*Topic 1
- BADREVIEW GOODREVIEW just like place dont im know tabl review
*Topic 2
- BADREVIEW chicken fri like GOODREVIEW tast order just food sauc
*Topic 3
- GOODREVIEW BADREVIEW bbq good place rib sauc brisket great meat
*Topic 4
- GOODREVIEW taco food BADREVIEW good mexican place salsa great chip
*Topic 5
- GOODREVIEW BADREVIEW food alway servic good time locat place love
*Topic 6
- GOODREVIEW BADREVIEW bar place drink great good food game night
*Topic 7
- GOODREVIEW BADREVIEW food place good chicken rice veri order great
*Topic 8
- BADREVIEW GOODREVIEW order time ask custom just said food minut
*Topic 9
- GOODREVIEW BADREVIEW food order servic wait tabl time server ask
*Topic 10
- GOODREVIEW breakfast BADREVIEW food good place great egg servic veri
*Topic 11
- GOODREVIEW sushi roll BADREVIEW place good great fish veri fresh
*Topic 12
- GOODREVIEW pizza good BADREVIEW place great veri italian cr

In [49]:
%%time
tf_final_200, vocab_final_200 = get_codewords(200)
model_200 = fit_and_print(tf_final_200, vocab_final_200)

300
300


KeyboardInterrupt: 

# Review Level Analysis

In [5]:
# read in yelp review dataframe
yelp_AZ = open_pickle('../data/yelp/yelp_reviews_phoenix.pkl')

In [6]:
yelp_AZ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164149 entries, 0 to 164148
Data columns (total 9 columns):
_id            164149 non-null object
business_id    164149 non-null object
date           164149 non-null object
review_id      164149 non-null object
stars          164149 non-null int64
text           164149 non-null object
type           164149 non-null object
user_id        164149 non-null object
votes          164149 non-null object
dtypes: int64(1), object(8)
memory usage: 12.5+ MB


# merge reviews to inspection event ids
cols_h = ['business_id', 'inspec_id','id_', 'date','date_start','sum_priority']
cols_y = []
rev_AZ = pd.merge(df_AZ_clean[cols_h], yelp_AZ, on='business_id', how='left', suffixes=['','_y'])

rev_AZ.info()

In [7]:
%%time
# Calculate term frequency matrix
tfr, tfr_vector = lib_tfidf.yelp_tf(yelp_AZ, 'text', max_features=10000)

CPU times: user 6min, sys: 8.31 s, total: 6min 8s
Wall time: 6min 7s


## Get Top +/- words

In [12]:
%%time
tfr_vocab = tfr_vector.get_feature_names()
tfr_pos  = tfr[:, np.array([x in set(pos_stem) for x in tfr_vocab]).nonzero()[0]]
tfr_neg  = tfr[:, np.array([x in set(neg_stem) for x in tfr_vocab]).nonzero()[0]]
vocabr_pos = [x for x in tfr_vocab if x in set(pos_stem)]
vocabr_neg = [x for x in tfr_vocab if x in set(neg_stem)]

CPU times: user 11.2 s, sys: 266 ms, total: 11.4 s
Wall time: 11.7 s


In [13]:
#print top occuring positive words (only count document frequency)
docr_pos = (tfr_pos > 0).sum(axis=0).tolist()[0]
indr_pos = np.argsort(docr_pos)[::-1]
topr_pos = [vocabr_pos[x] for x in indr_pos]

In [14]:
#print top occuring positive words (only count document frequency)
docr_neg = (tfr_neg > 0).sum(axis=0).tolist()[0]
indr_neg = np.argsort(docr_neg)[::-1]
topr_neg = [vocabr_neg[x] for x in indr_neg]

## LDA Codeword

In [21]:
%%time
tfr_final_300, vocabr_final_300 = get_codewords(300, tf=tfr, tf_vector=tfr_vector, top_pos=topr_pos, top_neg=topr_neg)
modelr_300 = fit_and_print(tfr_final_300, vocabr_final_300)

300
300
*Topic 0
- GOODREVIEW BADREVIEW chicken food rice thai good order dish place
*Topic 1
- GOODREVIEW BADREVIEW coffe ice place tri good love cream tea
*Topic 2
- GOODREVIEW pizza good BADREVIEW crust place italian order sauc salad
*Topic 3
- GOODREVIEW BADREVIEW time alway food place ive locat ha servic
*Topic 4
- GOODREVIEW BADREVIEW chicken bbq good sauc pork fri wing rib
*Topic 5
- GOODREVIEW BADREVIEW steak dinner great veri servic restaur meal night
*Topic 6
- GOODREVIEW BADREVIEW fish shrimp good fri tri veri seafood place
*Topic 7
- GOODREVIEW taco salsa mexican good burrito chip BADREVIEW food bean
*Topic 8
- BADREVIEW GOODREVIEW food like just tast place good bad star
*Topic 9
- GOODREVIEW food good great veri servic place friendli staff price
*Topic 10
- GOODREVIEW BADREVIEW order ask time manag custom servic said food
*Topic 11
- GOODREVIEW sandwich salad good lunch place BADREVIEW chicken bread order
*Topic 12
- GOODREVIEW great food place love servic amaz best recomm

In [26]:
%%time
tfr_final_200, vocabr_final_200 = get_codewords(200, tf=tfr, tf_vector=tfr_vector, top_pos=topr_pos, top_neg=topr_neg)
modelr_200 = fit_and_print(tfr_final_200, vocabr_final_200)

200
200
*Topic 0
- GOODREVIEW chicken food good BADREVIEW veri lunch place salad tri
*Topic 1
- GOODREVIEW food great good servic veri place friendli staff price
*Topic 2
- GOODREVIEW BADREVIEW salad steak veri order dish dessert dinner meal
*Topic 3
- GOODREVIEW pizza good sauc crust BADREVIEW order italian wing place
*Topic 4
- GOODREVIEW breakfast BADREVIEW egg coffe good place pancak bacon great
*Topic 5
- GOODREVIEW BADREVIEW chicken thai order good rice food dish soup
*Topic 6
- GOODREVIEW BADREVIEW restaur locat place menu area ha park food
*Topic 7
- GOODREVIEW taco salsa mexican good BADREVIEW chip burrito food bean
*Topic 8
- GOODREVIEW food great servic veri time restaur experi server staff
*Topic 9
- GOODREVIEW BADREVIEW beer bar good place great drink like food
*Topic 10
- BADREVIEW GOODREVIEW order time ask custom manag food servic said
*Topic 11
- GOODREVIEW BADREVIEW burger fri good order dog chees onion hot
*Topic 12
- GOODREVIEW BADREVIEW good bbq chicken sauc pork ch

In [30]:
doc_topic = modelr_200.doc_topic_
for n in range(10):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n{}...".format(n,
                                            topic_most_pr,
                                            yelp_AZ.loc[n]['text']))

doc: 0 topic: 3
never has crushed peppers! 

but the pizza ain't too shabby....
doc: 1 topic: 19
The pizza wasn't horrible,  but the deal was awful.  The prices are extremely high at Dominos in general,  and their medium 2 topping is a rip off.  Low rating is for price vs.  Quality,  and value....
doc: 2 topic: 10
I normally don't have a problem with Dominos--you pretty much know what you're getting for your money.

Unfortunately, this Dominos, even though it is less than 3 miles away, won't deliver to my address. Not only will they not deliver, but they will take my order, ask me what the cross-streets are (so they KNOW how far I am), and wait 30 minutes before calling to tell me that they cancelled my order.

They could have checked my address when I gave it to them, or they could have called and asked if I wanted to pick it up instead, but instead they simly cancelled my order--and didn't let me know until it was supposed to be at my door,...
doc: 3 topic: 19
I don't usually write r

In [32]:
%%time
tfr_final_200, vocabr_final_200 = get_codewords(200, tf=tfr, tf_vector=tfr_vector, top_pos=topr_pos, top_neg=topr_neg)
modelr_200 = fit_and_print(tfr_final_200, vocabr_final_200, alpha=1./20, eta=1./20)

200
200
*Topic 0
- GOODREVIEW BADREVIEW chicken food good thai rice place order dish
*Topic 1
- GOODREVIEW food great servic good veri place friendli love staff
*Topic 2
- GOODREVIEW breakfast BADREVIEW egg good place pancak bacon food great
*Topic 3
- GOODREVIEW BADREVIEW salad menu dish wine order restaur veri flavor
*Topic 4
- GOODREVIEW BADREVIEW ice coffe place love cream tri tea good
*Topic 5
- GOODREVIEW pizza good BADREVIEW place crust order sauc great italian
*Topic 6
- GOODREVIEW BADREVIEW place food veri good great nice seat restaur
*Topic 7
- GOODREVIEW taco salsa BADREVIEW good mexican food chip burrito place
*Topic 8
- GOODREVIEW great BADREVIEW veri steak servic food dinner good order
*Topic 9
- GOODREVIEW beer BADREVIEW great bar place good drink happi hour
*Topic 10
- GOODREVIEW BADREVIEW order time food custom ask manag servic said
*Topic 11
- GOODREVIEW BADREVIEW burger fri good dog order place chees like
*Topic 12
- GOODREVIEW BADREVIEW good bbq chicken sauc pork ri

In [33]:
1./20

0.05

In [56]:
def get_probability_score(topic_word):
    K, W = topic_word.shape
    
    avg_log_score = np.log(topic_word).sum(axis=0)/K
    prob_score = np.multiply(topic_word, np.log(topic_word) - avg_log_score)
    
    return  prob_score

In [62]:
print_topics(modelr_200.topic_word_, vocabr_final_200, n=10)

*Topic 0
- thai rice chicken pho chines noodl curri soup dish spici
*Topic 1
- GOODREVIEW great food gyro servic pita friendli veri love hummu
*Topic 2
- breakfast egg pancak coffe bacon toast brunch morn hash biscuit
*Topic 3
- wine salad dish chees dessert pork flavor cocktail bread appet
*Topic 4
- coffe ice tea cream vegan chocol donut dessert GOODREVIEW gluten
*Topic 5
- pizza crust italian wing sauc pasta chees salad sausag pepperoni
*Topic 6
- seat park patio tabl outsid GOODREVIEW outdoor BADREVIEW locat great
*Topic 7
- taco salsa mexican burrito chip bean enchilada tortilla carn margarita
*Topic 8
- steak GOODREVIEW dinner birthday lobster great filet server dessert crab
*Topic 9
- beer bar great hour GOODREVIEW game drink happi bartend music
*Topic 10
- BADREVIEW manag order custom card ask said rude told phone
*Topic 11
- burger fri BADREVIEW dog onion bun chees shake ring bacon
*Topic 12
- bbq rib pork sauc chicken brisket fri potato BADREVIEW mac
*Topic 13
- sandwich sala

In [63]:
print_topics(modelr_300.topic_word_, vocabr_final_200, n=10)

*Topic 0
- thai chicken rice pho soup chines noodl dish curri roll
*Topic 1
- coffe ice tea cream vegan buffet GOODREVIEW dessert indian pasti
*Topic 2
- pizza crust italian pasta salad wing chees sauc slice pie
*Topic 3
- alway locat time ive year BADREVIEW food ha servic love
*Topic 4
- bbq chicken rib wing pork sauc fri brisket chees mac
*Topic 5
- steak dinner dessert birthday rib great wine salad night filet
*Topic 6
- fish shrimp fri seafood crab BADREVIEW sauc lobster oyster clam
*Topic 7
- taco salsa mexican burrito chip bean tortilla enchilada margarita carn
*Topic 8
- BADREVIEW bad tast food bland star ok price wasnt dri
*Topic 9
- GOODREVIEW great food servic good veri friendli staff price clean
*Topic 10
- manag BADREVIEW order custom ask told said card rude minut
*Topic 11
- sandwich salad chicken pita gyro bread bagel sub lunch soup
*Topic 12
- great GOODREVIEW love amaz servic food atmospher staff wine place
*Topic 13
- sushi roll hour happi tuna price GOODREVIEW fish gr